In [365]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle

# Why

## Motivation
We want model to output classes rather than meaningless floating point values

<img src="why.jpeg">

# How

## Math

$$
\begin{aligned}
\text{odds($p$)} &= \frac{p}{1-p}\\ 
\text{log-odds($p$)} &= log(\frac{p}{1-p})
\end{aligned}
$$

$$
\begin{aligned}
t &= log(\frac{p}{1-p})\\ 
e^{t} &= \frac{p}{1-p}\\
e^{t} - pe^{t} &= p\\
p &= \frac{e^{t}}{1+e^{t}} = \frac{1}{1+e^{-t}}, t = x^{T}\theta\\
P(Y = 1 | x) &= \frac{1}{1 + e^{-x^{T}\theta}} = \sigma (x^{T}\theta)\\
\end{aligned}$$

This just looks like a linear model, so we wrap $\sigma$ around it and call LR a GLM, because it is a non-linear transformation of a linear model.

### Properties of logistic function

Definition: $\sigma(t) = \frac{1}{1+e^{-t}} = \frac{e^t}{1+e^t}$  
Range: $0 < \sigma(t) < 1$  
Inverse: $t = \sigma^{-1}(p) = \log\left(\frac{p}{1-p}\right)$  
Reflection and Symmetry: $1 - \sigma(t) = \frac{e^{-t}}{1+e^{-t}} = \sigma(-t)$  
Derivative: $\frac{d}{dt} \sigma(t) = \sigma(t)(1-\sigma(t)) = \sigma(t) \sigma(-t)$


### Parameter interpretation

$$
\begin{aligned}
t &= log(\frac{p}{1-p})\\ 
e^{x^{T}\theta} &= \frac{P(Y = 1|x)}{P(Y = 0|x)}\\
\end{aligned}
$$

### Loss function choice

**MSE is no good because**
Loss is not convex
<img src="mse.jpeg">
and loss is not large enough to penalize
<img src="mse_penalize.jpeg">

**Cross-entropy** comes to rescue. Negative log loss can goes to infinity on wrong predictions.
<img src="nll.jpeg">

$$
\text{loss} = 
\begin{cases} 
-\log(1-\hat{y}) & \text{if } y = 0 \\
-\log(\hat{y}) & \text{if } y = 1
\end{cases}
$$
$$
\text{loss} = -y log(\hat{y}) - (1-y)log(1-\hat{y})
$$



### Softmax and Cross-Entropy Loss Derivation

#### **Softmax Function Definition:**

The softmax function is defined as follows:

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{k=1}^K e^{z_k}}
$$

where $( z_i )$ is the logit corresponding to the $( i )$-th class and $( K )$ is the total number of classes.

#### **Cross-Entropy Loss Definition:**

The cross-entropy loss for a multi-class classification problem is defined as:

$$
L = -\sum_{i=1}^K y_i \log(\hat{p}_i)
$$

where $( y_i )$ is the true label in one-hot encoded format $( y_i = 1 )$ for the correct class and 0 otherwise, and $( \hat{p}_i )$ is the predicted probability for class $( i )$, given by the softmax of $( z_i )$.

#### **Gradient Derivation:**

We need to find how the loss $( L )$ changes with each logit $( z_j )$. This involves using the chain rule for partial derivatives:

$$
\frac{\partial L}{\partial z_j} = \sum_{i=1}^K \frac{\partial L}{\partial \hat{p}_i} \frac{\partial \hat{p}_i}{\partial z_j}
$$

The partial derivatives are calculated as follows:

- **Loss with respect to the predicted probabilities:**

$$
\frac{\partial L}{\partial \hat{p}_i} = -\frac{y_i}{\hat{p}_i}
$$

- **Predicted probabilities with respect to logits:**

$$
\begin{aligned}
\frac{\partial}{\partial z_j}log(\hat{p}_i) &= \frac{1}{\hat{p}_i} \cdot \frac{\partial \hat{p}_i}{\partial z_j}\\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot \frac{\partial}{\partial z_j}log(\hat{p}_i) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot \frac{\partial}{\partial z_j}log(\frac{e^{z_i}}{\sum_{k=1}^K e^{z_k}}) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot \frac{\partial}{\partial z_j}(log(e^{z_i}) - log(\sum_{k=1}^K e^{z_k})) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot \frac{\partial}{\partial z_j}(z_i - log(\sum_{k=1}^K e^{z_k})) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot (\frac{\partial z_i}{\partial z_j} - \frac{\partial}{\partial z_j}log(\sum_{k=1}^K e^{z_k})) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot (\frac{\partial z_i}{\partial z_j} - (\frac{1}{\sum_{k=1}^K e^{z_k}})\frac{\partial}{\partial z_j}(\sum_{k=1}^K e^{z_k})) \\
\frac{\partial \hat{p}_i}{\partial z_j} &= \hat{p}_i \cdot (\frac{\partial z_i}{\partial z_j} - (\frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}})) \\
\end{aligned}
$$

For $( i = j )$:

$$
\frac{\partial \hat{p}_i}{\partial z_j} = \hat{p}_i (1 - \hat{p}_j)
$$

For $( i \neq j )$:

$$
\frac{\partial \hat{p}_i}{\partial z_j} = -\hat{p}_i \hat{p}_j
$$

#### **Combining These Results:**

Combine the partial derivatives into the final gradient formula:

$$
\begin{aligned}
\frac{\partial L}{\partial z_j} &= \sum_{i=1}^K -\frac{y_i}{\hat{p}_i} \frac{\partial \hat{p}_i}{\partial z_j}
\end{aligned}
$$

$$
\frac{\partial L}{\partial z_j} = -\frac{y_j}{\hat{p}_j} \hat{p}_j (1 - \hat{p}_j) + \sum_{i \neq j} -\frac{y_i}{\hat{p}_i} (-\hat{p}_i \hat{p}_j)
$$

$$
\frac{\partial L}{\partial z_j} = -y_j (1 - \hat{p}_j) + \sum_{i \neq j} y_i \hat{p}_j
$$

$$
\frac{\partial L}{\partial z_j} = -y_j + y_j \hat{p}_j + \hat{p}_j \sum_{i \neq j} y_i = \hat{p}_j - y_j
$$

because $( \sum_{i=1}^K y_i = 1 )$ (since $( y_i )$ is one-hot encoded).

#### **Final Gradient Result:**

$$
\frac{\partial L}{\partial z_j} = \hat{p}_j - y_j
$$

This gradient indicates the direction and magnitude by which the logits $( z_j )$ need to be adjusted to minimize the loss, facilitating the update of weights during training.


## Implementation

In [284]:
import time

In [957]:
class LR:
    def __init__(
            self, max_iter=100, learning_rate=0.01, 
            decision_thres=0.5, regularization='l2', 
            lambda_reg=0.01, batch_size=32, momentum=0.9,
            random_state=None
        ):
        self.decision_thres = decision_thres
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.regularization = regularization
        self.lambda_reg = lambda_reg
        self.weights = None
        self.bias = None
        self.batch_size = batch_size
        self.momentum = momentum
        self.random_state = random_state

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _softmax(self, z):
        # Stable softmax function
        exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    def loss(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-16, 1 - 1e-16)
        base_loss = (-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred)).mean()
        if self.regularization == 'l2':
            reg_loss = 0.5 * self.lambda_reg * np.sum(self.weights ** 2)
        elif self.regularization == 'l1':
            reg_loss = self.lambda_reg * np.sum(np.abs(self.weights))
        else:
            reg_loss = 0
        return base_loss + reg_loss

    def _derivative_ce_loss(self, y, z):
        sigma_z = self._softmax(z)
        grad = sigma_z - y  # The simplified derivative of loss with respect to z
        return grad

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = np.max(y) + 1
        if self.random_state:
            np.random.seed(self.random_state)
        self.weights = np.random.normal(scale=0.1, size=(n_features, n_classes))
        self.bias = np.random.normal(scale=0.1, size=n_classes)

        v_weights = np.zeros((n_features, n_classes))
        v_bias = np.zeros(n_classes)
        
        indices = list(range(n_samples))
        np.random.shuffle(indices)
        indices = cycle(indices)

        for i in range(self.max_iter):
            if self.batch_size:
                batch = [next(indices) for _ in range(self.batch_size)]
                X_batch, y_batch = X[batch], y[batch]
            else:
                X_batch, y_batch = X, y
            
            logits = X_batch @ self.weights + self.bias
            dloss = self._derivative_ce_loss(np.eye(n_classes)[y_batch], logits)
            dw = X_batch.T @ dloss / n_samples
            db = dloss.mean()
    
            # Regularization updates
            if self.regularization == 'l2':
                dw += self.lambda_reg * self.weights
            elif self.regularization == 'l1':
                dw += self.lambda_reg * np.sign(self.weights)

            v_weights = self.momentum * v_weights + self.learning_rate * dw
            v_bias = self.momentum * v_bias + self.learning_rate * db

            self.weights -= v_weights
            self.bias -= v_bias

    def predict_prob(self, X):
        logits = X @ self.weights + self.bias
        return self._softmax(logits)

    def predict(self, X):
        return np.argmax(self.predict_prob(X), axis=1)

    def score(self, X, y):
        return (self.predict(X) == y).mean()


In [958]:
def softmax(z):
    # Stable softmax function
    exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

np.random.seed(0)  # For reproducibility

# Generate random data
num_samples = 1000000
num_features = 10
num_classes = 50

# Random features
X = np.random.randn(num_samples, num_features)

# Generate labels based on a linear combination of features with noise
weights = np.random.randn(num_features, num_classes)  # Arbitrary weights for features
bias = np.random.randn(num_classes)  # Some bias
noise = np.random.normal(0, 0.05, (num_samples, num_classes))  # Noise to make the task not perfectly linear

# Linear combination to generate continuous values
linear_combination = X.dot(weights) + bias + noise

probabilities = softmax(linear_combination)

# Binarize probabilities to get binary labels
# threshold = 0.5
# labels = (probabilities > threshold).astype(int)
labels = np.argmax(probabilities, axis=1)

# Display the first few samples
print("Features (first 5 samples):\n", X[:5])
print("Labels (first 5 samples):", labels[:5])

indices = list(range(num_samples))
np.random.shuffle(indices)
X_train, X_test = X[indices[:int(num_samples*0.8)]], X[indices[int(num_samples*0.8):]]
labels_train, labels_test = labels[indices[:int(num_samples*0.8)]], labels[indices[int(num_samples*0.8):]]

Features (first 5 samples):
 [[ 1.76405235  0.40015721  0.97873798  2.2408932   1.86755799 -0.97727788
   0.95008842 -0.15135721 -0.10321885  0.4105985 ]
 [ 0.14404357  1.45427351  0.76103773  0.12167502  0.44386323  0.33367433
   1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502  2.26975462 -1.45436567
   0.04575852 -0.18718385  1.53277921  1.46935877]
 [ 0.15494743  0.37816252 -0.88778575 -1.98079647 -0.34791215  0.15634897
   1.23029068  1.20237985 -0.38732682 -0.30230275]
 [-1.04855297 -1.42001794 -1.70627019  1.9507754  -0.50965218 -0.4380743
  -1.25279536  0.77749036 -1.61389785 -0.21274028]]
Labels (first 5 samples): [48 37 45 25 32]


In [981]:
lr = LR(max_iter=100, learning_rate=2, lambda_reg=0.0001, batch_size=20480, momentum=0.9, random_state=0)

In [982]:
time_start = time.time()
lr.fit(X_train, labels_train)
time_end = time.time()

print(f"{time_end-time_start} seconds")

2.189581871032715 seconds


In [983]:
lr.score(X_test, labels_test)

0.679065

In [968]:
from sklearn.linear_model import SGDClassifier

In [954]:
sklearn_lr = SGDClassifier(max_iter=50, random_state=0, n_jobs=-1)

In [955]:
time_start = time.time()
sklearn_lr.fit(X_train, labels_train)
time_end = time.time()

print(f"{time_end-time_start} seconds")

5.057495832443237 seconds


In [956]:
sklearn_lr.score(X_test, labels_test)

0.66023